# CNN 卷积神经网络

MLP为了将像素输出Dense层而进行了展平操作，将二维网格强制拉成了一维，因此丧失了空间信息/关系。

```python
# MLP处理图像的典型流程
input_image = (32, 32, 3)  # 高32, 宽32, 3通道的彩色图像
flattened = Flatten()(input_image)  # 变成3072维向量 (32*32*3=3072)
dense_layer = Dense(128)(flattened) # 全连接层

而CNN直接处理二维结构

```python
input_image = (32, 32, 3)
conv_layer = Conv2D(filters=32, kernel_size=(3, 3), padding='same')(input_image)
# 输出保持空间维度: (32, 32, 32)

## MLP vs CNN
输入图像A (笑脸在左侧):<br>
[:), ., ., .]<br>
[ . , ., ., .]<br>
[ . , ., ., .]<br>
[ . , ., ., .]<br>

输入图像B (笑脸在右侧):<br>
[., ., ., .]<br>
[., ., ., .]<br>
[., ., ., .]<br>
[., ., ., :)]<br>

展平后：
图像A: [':)', '.', '.', '.', '.', '.', '.', '.', ...] 
图像B: ['.', '.', '.', '.', '.', '.', '.', '.', ... , ':)']
→ MLP认为这是两个完全不同的输入


同一个卷积核滑动检测：<br>
位置1: 覆盖 <br>
         [:), ., .]   → 检测到笑脸 → 激活<br>
         [ . , ., .]<br>
         [ . , ., .]<br>
         
位置2: 覆盖 <br>
         [., ., .]     → 未检测到 → 不激活<br>
         [., ., .]<br>
         [., ., .]<br>
         
位置N: 覆盖<br>
         [., ., .]     → 检测到笑脸 → 激活<br>
         [., ., .] <br>
         [., ., :)]<br>

CNN知道这是同一个特征在不同位置

## CNN三大设计原则
1. 局部感受野
- 每个卷积核只看到输入的一小部分
- 3×3卷积核只连接9个输入像素，而不是全部像素
2. 权重共享
- 同一个卷积核扫过整个图像
- 无论检测左上角的边缘还是右下角的边缘，都用相同的权重
3. 层次化特征学习
- 通过多层卷积构建特征层次<br>
$ Conv2D(64, (3,3)) → Conv2D(128, (3,3)) → Conv2D(256, (3,3)) $
- 边缘 → 纹理 → 部件 → 物体

## Strides（步幅）

步幅是卷积核在输入数据上滑动时的移动步长

**特点**：
- **步幅=1**：卷积核每次移动1个像素，感受野重叠较多，输出尺寸较大
- **步幅>1**：卷积核跳跃移动，感受野重叠较少，输出尺寸较小
- 常用值：1、2；
- *通常使用该参数减少张量通过神经网络时占用的空间大小，同时增加通道的数量*

**作用**：控制输出特征图的尺寸和下采样程度

**计算公式**
输出尺寸 = (输入尺寸 - 核大小 + 2×填充) / 步幅 + 1


## Padding（填充）
在输入数据的边缘添加额外像素（通常是0）

**类型**：
- **Valid Padding**（无填充）
  - 不添加任何填充
  - 输出尺寸会缩小
- **Same Padding**（相同填充）
  - 添加填充使输出尺寸与输入尺寸相同
  - 填充大小 = (核大小 - 1) / 2

**作用**：
- 控制输出特征图的尺寸
- 防止边缘信息丢失
- 保持空间维度

## Kernel（卷积核）

**定义**：在输入数据上滑动的小型滤波器，用于提取特征
卷积核是一个小的权重矩阵，在输入数据上滑动以提取特征。

```python
# 示例：3x3 卷积核
kernel = [[w11, w12, w13],
          [w21, w22, w23],
          [w31, w32, w33]]
```

**特性**：
- **尺寸**：通常是正方形（3×3、5×5等）
- **深度**：与输入通道数相同
- **数量**：决定输出特征图的通道数

**功能**：
- 提取局部特征（边缘、纹理、形状等）
- 通过权重共享减少参数数量
- 不同的核学习不同的特征模式

**这三个参数共同决定了卷积层的输出维度和特征提取能力。**

在神经网络训练中，随着网络层数加深，输入分布会不断变化（Internal Covariate Shift），造成训练不稳定、收敛慢、甚至梯度爆炸/消失。
**所以在每层的输入上做规范化，使其具有“稳定且一致”的统计特性。**

---

## **批标准化（Batch Normalization, BN）**

BN在 **batch 维度上** 做归一化。

也就是说：

> 对一个 mini-batch 中，同一个特征维度的所有样本，统计它们的均值和方差，再根据这个均值与方差进行归一化。

---

### **计算流程**

假设输入张量形状为：
$
x \in \mathbb{R}^{B \times C \times H \times W}
$

BN对每个通道 ( C ) 独立计算：

### **计算批均值与方差**

$
\mu_c=\frac{1}{BHW}\sum_{b,h,w} x_{b,c,h,w}
$

$
\sigma_c^2=\frac{1}{BHW}\sum_{b,h,w}(x_{b,c,h,w}-\mu_c)^2
$

### **归一化**

$
\hat x_{b,c,h,w}=\frac{x_{b,c,h,w}-\mu_c}{\sqrt{\sigma_c^2+\epsilon}}
$

### **缩放 + 平移（可学习参数）**

$
y_{b,c,h,w}=\gamma_c \hat x_{b,c,h,w}+\beta_c
$

其中 $(\gamma_c, \beta_c)$ 为每个通道的可学习参数，改善表达能力。

---
**优点**

* 训练非常稳定（加速梯度传播）
* 能使用更大的学习率
* 加快收敛速度
* 对 CNN 效果拔群（尤其是图像）

---

**缺点**

* **依赖 batchsize**（小批量或 batch=1 时性能差）
* RNN/transformer 不适用（序列长度可变、batch 维度乱序）
* 训练和推理阶段采用 **不同统计量**（running mean/var 与 batch mean/var）



---

### **层标准化（Layer Normalization, LN）**

LN在 **特征维度（channels 或 hidden dimension）** 上做归一化。

它不依赖 batch，而是对 **每个样本自己做 normalize**。

---

## **计算流程**

假设在 Transformer 中：
$
x \in \mathbb{R}^{B \times L \times D}
$
每个 token 的 hidden size 是 (D)。

对每个样本的每个 token：

### **计算特征维度上的均值与方差：**

$
\mu = \frac{1}{D}\sum_d x_d
$

$
\sigma^2=\frac{1}{D}\sum_d (x_d - \mu)^2
$

### **归一化：**

$
\hat x = \frac{x - \mu}{\sqrt{\sigma^2+\epsilon}}
$

### **缩放与平移（与 BN 类似）：**

$
y = \gamma \hat x + \beta
$

其中 $(\gamma, \beta)$ 对每个特征维度学习一个 scalar。

---

**优点**

* **与 batchsize 无关**，可 batch=1
* 非常适合 **RNN / Transformer / NLP**
* 对序列模型效果稳定
* 训练/推理一致（统计值只来自输入本身）

---

**缺点**

* 在 CNN 中效果一般（因为跨通道归一化会破坏空间结构）
* 在大 batch 的图像任务中性能不如 BN

---



## **BN 与 LN 的最核心区别总结**

| 特性                      | BatchNorm         | LayerNorm     |
| ----------------------- | ----------------- | ------------- |
| **归一化维度**               | 跨 batch，跨空间，对每个通道 | 对每个样本内部的特征维度  |
| **依赖 batch size**       | 是                 | 否             |
| **适用于 CNN**             | 非常好               | 一般            |
| **适用于 RNN/Transformer** | 差                 | 非常好           |
| **训练/推理阶段是否一致**         | 不一致               | 完全一致          |
| **计算均值/方差位置**           | batch + H/W       | 仅 feature dim |
| **推理阶段统计方式**            | running mean/var  | 无需 running 统计 |

---

## 为什么 Transformer 必须用 LayerNorm？

Transformer 中：

* 序列长度 L 变化大
* batchsize 经常很小（甚至 1）
* attention 操作会破坏 batch 维度一致性

BN 会导致：

* 统计量不稳定
* batch 内 token 混合破坏顺序

而 LN：

* 每个 token 独立规范化
* 与 batchsize 无关
* 训练更稳定

因此 Transformer 一律使用 LayerNorm。

---

## 图形化直观理解

### **BN：跨样本归一化**

```
Batch: 8 samples
Feature: channel 1
Compute mean/var across 8 samples → normalize
```

### **LN：每个样本内部归一化**

```
Sample: 1
Feature vector: 256-dim
Compute mean/var across 256 dims → normalize
```

---

* **BN 是“跨样本规范化”，适合计算机视觉；**
* **LN 是“单样本内部规范化”，适合 NLP、RNN、Transformer。**


## **Dropout 的核心思想**

**随机“丢弃”一部分神经元（让其输出变为 0）用来减少过拟合。**

在训练时，Dropout 以一定概率 (p) 将神经网络中的部分激活值置为 0，**使模型不能过度依赖特定神经元**，从而提高泛化能力。

---

## **Dropout 的数学定义**

假设一层神经元输出为：

%
h = f(Wx + b)
$

Dropout 会产生一个与 (h) 同形状的随机掩码：

$
m_i \sim \text{Bernoulli}(1-p)
$

掩码示意：

* 以概率 (1-p) 保留
* 以概率 (p) 丢弃（置为 0）

应用 Dropout 后：

$
h_i^{drop} = \frac{m_i}{1-p} \cdot h_i
$

其中 $(\frac{1}{1-p})$ 是为了 **保持期望不变（inverted dropout）**，这样推理阶段就不需要额外缩放。

**期望检查：**

$
E[h_i^{drop}] = h_i
$

---

## **Dropout 的训练/推理两种模式**

**训练阶段（train）**

* 随机屏蔽神经元
* 保留概率 = (1-p)
* 输出缩放 (\frac{1}{1-p})

*网络每次都像“变成了一个子网络”，不同 batch 使用不同 mask。*

---

## **推理阶段（eval）**

* **不再进行丢弃**
* 不再使用 mask
* 不进行缩放，因为训练阶段已经做了期望保持

因此，Dropout 是一种只在训练时生效的正则化方法。

---


## **Dropout原理**

- 降低 co-adaptation（神经元互相“串通”）, 如果网络太强，特征之间可能会过度依赖：

> 某一类特征只在另一个特征存在时才有效。

- Dropout 随机屏蔽神经元，迫使网络 **每个神经元都要学到更通用、更健壮的特征**。

- 相当于对神经网络进行 "ensemble" 集成学习**

训练中：

* Dropout 每次随机选择一个子网络
* 所有子网络共享参数
* 最终模型 = **指数级数量的子模型集成**

-这也是它提升泛化能力的本质原因：

> “轻量级的模型集成方法”。

- **引入噪声，降低模型 variance**

- 在训练时加入随机噪声，使模型对小扰动更不敏感，从而提升泛化。

---

## **哪些地方通常使用 Dropout？**

**MLP 全连接层中效果明显**
如：

* 图像分类最后几层 FC
* NLP 中的 embedding 全连接层
* Tabular data 网络

---

**CNN 中的卷积层中不太常用**

因为：

* 卷积层本身参数共享，过拟合倾向弱
* 丢弃像素可能破坏局部空间相关性

一般只在全连接层使用 Dropout。

---

**Transformer 中大量使用 Dropout**

尤其：

* Attention 分数后的 dropout (attention dropout)
* FFN 层后的 dropout
* Residual connection 末尾的 dropout
* Embedding dropout

但 Transformer 更常用 **LayerNorm + Residual + Dropout**，改变了 Dropout 的作用位置。

---

## Dropout 的常见变种**

**1. SpatialDropout**

在 CNN 中，不是随机丢弃单个像素，而是丢弃整个 feature map（通道）。

---

**2. DropConnect**

丢弃权重，而不是丢弃激活值。

---

**3. ZoneOut（RNN）**

Dropout 在 RNN 中作用会破坏时间依赖，因此使用 ZoneOut 保留部分旧的 hidden state。

---

**4. Stochastic Depth（ResNet）**

整个残差分支随机 skip，相当于 Dropout 的深度版。

---

**5. DropPath（Transformer）**

Transformer 中的 Stochastic Depth，用于 Vision Transformer（ViT）和大型 Transformer。

---


### **在Transformer中的应用**

```python
self.dropout = nn.Dropout(p=0.1)
x = x + self.dropout(ffn_output)
```

---



# **Dropout 的优缺点**

| 操作              | 优点                                          | 缺点                              |
| --------------- | ------------------------------------------- | ------------------------------- |
| **Dropout**     | 强大的正则化作用；防止过拟合；训练简单；相当于大量模型集成               | 改变训练分布，训练变慢；对 CNN 效果一般；不适用于 RNN |
| **不使用 Dropout** | 快；显著提升 CNN 中卷积层表现；ResNet 等深层结构靠 BN/LN 提供稳定性 | 更容易过拟合                          |

---

> **Dropout 在训练时随机屏蔽神经元，使网络不能依赖单一特征，从而增强泛化，起到类似“集成学习”的效果，是最经典、最有效的正则化技术之一。**